# Loading Data

Replace "c" with the actual dataset you want to load.

In [245]:
load_data = 'c'

In [246]:
import pywt
import librosa
import pandas as pd
import pathlib
import glob
import os
import numpy as np
from tensorflow.data import Dataset


from scipy.io.wavfile import read
import matplotlib.pyplot as plt

In [247]:
data_dir = "/Users/sentissi/code/Merarztin/Waves_of_The_Heart/raw_data"
print(data_dir)
files_path = pathlib.Path(data_dir)
files_list = list(files_path.glob(f"training-{load_data}/*"))
filtered_list = [file for file in files_list if ".wav" in str(file)]

/Users/sentissi/code/Merarztin/Waves_of_The_Heart/raw_data


Ensure that the filter is working as intended

In [264]:
print(len(files_list))
print(len(filtered_list))

71
155


In [249]:
labels_data = pd.read_csv("/Users/sentissi/code/Merarztin/Waves_of_The_Heart/raw_data/annotations/Online_Appendix_training_set.csv")

In [250]:
# function for loading audio files
def load_audio_file(file_path):
    signal, sr = librosa.load(file_path, sr=None)
    return signal, sr

# Define data directory and file list
data_dir = f"/Users/sentissi/code/Merarztin/Waves_of_The_Heart/raw_data/training-{load_data}/"
file_list = os.listdir(data_dir)

# Filter files to only include .wav files
filtered_list = [file for file in file_list if ".wav" in str(file)]
filtered_list = list(np.repeat(filtered_list,5))
filtered_list = [s.replace('.wav', '') for s in filtered_list]

def extracting_wavelet_features(filepath):
    audio_signal, sr = librosa.load(filepath)
    
    # wavelet transform
    wavelet = pywt.Wavelet('db4')
    coefficients = pywt.wavedec(audio_signal, wavelet, level=4)
    
    # extracting features
    features = []
    for c in coefficients:
        duration = len(c) / sr  
        energy = np.sum(np.square(c))
        peak_amplitude = np.max(np.abs(c))
        mean_amplitude = np.mean(np.abs(c)) 
        std_amplitude = np.std(np.abs(c))
        skew_amplitude = np.abs(np.mean(c**3)) / np.power(np.mean(c**2), 3/2)
        kurtosis_amplitude = np.mean(c**4) / np.power(np.mean(c**2), 2)
        stats = {'Duration': duration, 'Energy': energy, 'Peak Amplitude': peak_amplitude,
                 'Mean Amplitude': mean_amplitude, 'Std Amplitude': std_amplitude,
                 'Skew Amplitude': skew_amplitude, 'Kurtosis Amplitude': kurtosis_amplitude}
        features.append(stats)
        
    return features

# Filter files to only include .wav files
audio_files = [os.path.join(data_dir, file) for file in os.listdir(data_dir) if file.endswith(".wav")]

list_features = []

for file in audio_files:
    list_features.append(extracting_wavelet_features(file))
        
df = pd.concat([pd.DataFrame(f) for f in list_features], ignore_index=True)
df['wav_file'] = filtered_list
df = df[['wav_file', 'Duration','Energy', 'Peak Amplitude', 'Mean Amplitude', 'Std Amplitude', 'Skew Amplitude', 'Kurtosis Amplitude']] 
df

,wav_file,Duration,Energy,Peak Amplitude,Mean Amplitude,Std Amplitude,Skew Amplitude,Kurtosis Amplitude
0,c0025,3.524444,1.085720e+04,3.975677,2.282524e-01,2.959864e-01,0.028460,12.457765
1,c0025,3.524444,4.771133e+01,0.398046,1.770939e-02,1.732952e-02,0.126698,11.241687
2,c0025,7.048571,9.672818e-01,0.044615,1.861510e-03,1.660844e-03,0.088091,11.262418
3,c0025,14.096871,6.665202e-03,0.010008,1.081563e-04,9.871694e-05,0.929554,90.436798
4,c0025,28.193424,5.812910e-05,0.004226,5.141902e-06,8.189404e-06,89.398347,71712.000561
...,...,...,...,...,...,...,...,...
150,c0028,0.914603,1.683894e+02,0.538697,6.452736e-02,6.469906e-02,0.263038,6.662409
151,c0028,0.914603,1.965726e-02,0.049826,1.999676e-04,9.668179e-04,1.866304,1092.016820
152,c0028,1.828889,5.258917e-04,0.006312,2.733903e-05,1.108750e-04,0.922036,821.575845
153,c0028,3.657460,3.893113e-06,0.000368,1.673278e-06,6.743414e-06,1.640111,706.750084


In [251]:
labels_data = labels_data.rename(columns={'Challenge record name': 'wav_file'})

In [252]:
#merged_df = pd.merge(df, labels_data, on='wav_file')
#merged_df = merged_df[["wav_file", "Duration", "Energy", "Peak Amplitude", "Mean Amplitude", "Std Amplitude", "Skew Amplitude", "Kurtosis Amplitude","Class (-1=normal 1=abnormal)"]]
#merged_df

In [253]:
lst_columns = []
for i in range(5):
    for column in df.columns:
        lst_columns.append(f'Decomp{i}_{column}')

In [256]:
lst_columns = []
for i in range(5):
    for column in df.columns:
        lst_columns.append(f'Decomp{i}_{column}')


In [257]:
final_df = pd.DataFrame(columns = lst_columns)
for wav_file in set(filtered_list):
    new_data = pd.DataFrame(df[df['wav_file'] == wav_file.split('.')[0]].to_numpy().reshape(1,-1), columns=lst_columns)
    final_df = pd.concat([final_df, new_data], ignore_index=True)

In [258]:
final_df = final_df.drop(['Decomp1_wav_file', 'Decomp2_wav_file', 'Decomp3_wav_file', 'Decomp4_wav_file'], axis=1)

In [259]:
final_df = final_df.rename(columns={'Decomp0_wav_file': 'wav_file'})

In [260]:
final_df['label'] = labels_data['Class (-1=normal 1=abnormal)']

In [261]:
final_df = final_df.rename(columns={'Decomp0_wav_file': 'wav_file'})

In [262]:
final_df

,wav_file,Decomp0_Duration,Decomp0_Energy,Decomp0_Peak Amplitude,Decomp0_Mean Amplitude,Decomp0_Std Amplitude,Decomp0_Skew Amplitude,Decomp0_Kurtosis Amplitude,Decomp1_Duration,Decomp1_Energy,...,Decomp3_Skew Amplitude,Decomp3_Kurtosis Amplitude,Decomp4_Duration,Decomp4_Energy,Decomp4_Peak Amplitude,Decomp4_Mean Amplitude,Decomp4_Std Amplitude,Decomp4_Skew Amplitude,Decomp4_Kurtosis Amplitude,label
0,c0022,4.488481,11875.213867,3.98117,0.194181,0.286846,0.064761,15.886394,4.488481,0.495228,...,0.707182,806.67174,35.905896,0.0,0.000059,0.0,0.000001,0.670631,848.286517,1
1,c0002,0.72381,209.600296,0.617345,0.080035,0.08202,0.665822,6.8043,0.72381,0.004075,...,9.125431,1728.556868,5.78839,0.0,0.000031,0.0,0.0,23.242434,4327.545282,1
2,c0021,3.772698,7417.644043,3.938824,0.190029,0.230339,0.146139,20.938793,3.772698,2.251711,...,32.15055,13198.523105,30.17941,0.000002,0.000829,0.000001,0.000002,137.514924,83848.631169,1
3,c0015,5.323764,37882.335938,3.419075,0.439433,0.360009,0.031178,3.746618,5.323764,7.990053,...,7.776329,2381.515168,42.587891,0.000004,0.00057,0.000001,0.000002,19.472953,6309.005213,1
4,c0010,5.581088,16049.868164,3.936494,0.223562,0.283619,0.312633,11.755677,5.581088,36.272514,...,0.009679,12.240332,44.646667,0.000016,0.000451,0.000002,0.000003,1.065306,197.69455,1
5,c0007,1.97873,9590.831055,3.722382,0.311095,0.350766,0.334948,9.759627,1.97873,3.320724,...,1.79478,212.857233,15.827664,0.000003,0.000325,0.000002,0.000002,4.679953,641.14878,1
6,c0001,1.420454,438.551758,1.08289,0.07231,0.093665,0.12177,14.223174,1.420454,0.003278,...,100.138573,42404.577345,11.361678,0.0,0.000208,0.0,0.000001,35.42282,67536.372246,-1
7,c0005,0.603356,183.891144,0.46975,0.093462,0.071324,0.115465,3.100394,0.603356,0.005867,...,86.690447,24421.514046,4.824671,0.000001,0.000955,0.0,0.000003,181.583546,56323.663188,1
8,c0023,2.250839,20060.035156,4.045582,0.346479,0.533044,0.639572,13.916684,2.250839,0.112613,...,1.082998,1607.748539,18.004671,0.000003,0.001312,0.0,0.000003,200.367859,186215.950966,-1
9,c0016,1.549932,9920.717773,3.963505,0.346109,0.412906,0.080014,11.262521,1.549932,1.134547,...,7.546732,1461.065892,12.397415,0.000001,0.000485,0.000001,0.000002,14.642273,11489.227773,1


Uncomment the cell bellow to save the file :)

In [263]:
# saving final_df
#final_df.to_csv(f'final_dataframe_from_{load_data}.csv', index=True)